Build a flow chart of a game <span style="background-color:orange"><b>with user interaction that uses all the file input/output features we covered in this model and everything else from previous ones</b></span>; YOU MUST VALIDATE ALL USER INPUT AND ALWAYS GIVE FRIENDLY ERROR MESSAGES. By this point, you might want to have some validation functions that you can use and re-use throughout the course; it is also important to have a control loop (outer loop) so that the player can quit the game when he/she wants; write the code for the game and explain its functionality; the game has to be functional without any error in order to get full points; explain the behavior of your code by writing plenty of good comments. Your game idea has to be unique. From now on, you MUST ALWAYS explain your code: documentation is crucial in programming and I want everyone to practice it, no matter how simple your code is; You can expand the game idea you posted on the discussion board if it was unique.

<h1>I selected the game of Go Fish</h1> (<i>which was a mistake once I realized how complex it actually was</i>)

Rules of the game:
  * Each player is dealt a hand of 7 cards.
  * If either player has a pair, they pull the pair from their hands and place them on the table.
  * One player goes first. The player picks a card in their hand that they’re trying to match, and they say, “Do you have a Jack?”
    * If the other player has a Jack, the asking player gets that card and places both Jacks on the table. The asking player goes again.
    * If the other player doesn’t have a Jack, they say “Go Fish!” The asking player draws a card from the deck and puts it in their hand.
      * If the drawn card is the card they asked for, they place the pair on the table and go again.
      * If the drawn card matches any other card in their hand, they place the pair on the table but don’t go again.
  * The game is over whenever one player runs out of cards.
  * The player with the most pairs at the end of the game wins, regardless of who ran out of cards first.

In [1]:
import os
import random

In [2]:
class Helper(object):

    @staticmethod
    def file_exists(file_path):
        """
        File exists check
        :param file_path: String full path to the file
        :return: Boolean value indicating if the file exists or not
        """
        return os.path.exists(file_path)
    
    @staticmethod
    def write_file(file_path, file_text):
        """
        Write file to disk
        :param file_path: String full path to the file
        :param file_text: String value to write to the file
        :return: Boolean value indicating the sucess of writting to the file
        """
        try:
            with open(file_path, 'w') as fh:
                fh.write(file_text)
        except IOError as e:
            print(f'Check the file path, {e.filename} could not be written to.')
            return False
        except TypeError as e:
            print(f'Check the type value of the file_text value: {file_text}')
            return False
        except:
            print('Generic write error')
            return False
        return True
                
    def read_file(self, file_path):
        """
        Read file from disk
        :param file_path: String Full path to the file
        :return: String value of the file contents or empty if file read fails
        """
        if not self.file_exists(file_path):
            print(f'Read file {file_path} does not exist')
            return ''
        try:
            with open(file_path, 'r') as fh:
                file_text = fh.read()
        except IOError as e:
            print(f'Check the file path, {e.filename} could not be read.')
            return ''
        except:
            print('Generic write error')
            return ''        
        return file_text
    
    @staticmethod
    def validated_input(input_prompt='Enter your name:', target_type=str, valid_options=[]):
        """
        Validated input wrapper method
        :param input_prompt: String used for the input prompt
        :param target_type: Python Type expected for the user input
        :param valid_options: List containing valid input options, defaults to empty list and is not applied
        :return: String of validated user input
        """
        # Confirm that target_type is actually a Type
        if not isinstance(target_type, type):
            print(f'Invalid target_type value {target_type}, not a Python Type')
        # Loop forever waiting for the user to enter the correctly formatted input
        while True:
            # Get the user input
            user_input = input(input_prompt)
            try:
                # Try casting the user input to the target type
                user_input_typed = target_type(user_input)
                # If there is a list of valid options enforce user input membership
                if len(valid_options) > 0 and user_input_typed not in valid_options:
                    valid_options_str = ', '.join([str(vo) for vo in valid_options])
                    print(f'Invalid input, must be one of the following values: {valid_options_str}')
                    continue
                break
            except:
                print(f'Invalid input {user_input} is not of type {target_type.__name__}')
        return user_input

In [3]:
class Card(object):
    def __init__(self, suit, value):
        """
        Card constructor
        :param suit: Suit string from Card.available_suits()
        :param value: Value string from Card.available_values()
        """
        assert suit in self.available_suits(), f'{suit} not an available suit'
        assert value in self.available_values(), f'{value} not an available value'
        self.suit = suit
        self.value = value

    @staticmethod
    def available_suits():
        """
        Available suits
        :return: List of available suit strings
        """
        return ['Clubs', 'Diamonds', 'Hearts', 'Spades']

    @staticmethod
    def available_values():
        """
        Available values
        :return: List of available values strings
        """
        return ['Ace', 'Jack', 'Queen', 'King'] + [str(c) for c in range(2, 11)]

    def is_value_match(self, card):
        """
        Check if the value of the current matches the card parameter
        :param card: Card object
        :return: Boolean value indicating if the cards match
        """
        return self.value == card.value

    def __repr__(self):
        return f'{self.value} of {self.suit}'


class Deck(object):
    def __init__(self):
        """
        Deck constructor for a standard 52 playing card deck
        """
        self.cards = []
        # Build the deck by iterating over the suits and values, adding each card combination
        for suit in Card.available_suits():
            for value in Card.available_values():
                self.cards.append(Card(suit=suit, value=value))

    def undealt_cards(self):
        """
        Undeal cards
        :return: Integer value for the number of undealt cards
        """
        return len(self.cards)

    def shuffle(self):
        """
        Shuffle the deck at random
        :return: N/A
        """
        random.shuffle(self.cards)

    def draw(self):
        """
        Draw a card from the top of the deck
        :return: Card object
        """
        return self.cards.pop(0)

    def deal(self, n_cards=1):
        """
        Deal the deck
        :param n_cards: Integer number of cards to deal
        :return: List of Card objects
        """
        cards_to_deal = []
        # Check to make sure enough cards are left to satisfy the request number of cards to be dealt
        if n_cards > self.undealt_cards():
            print(f'Requested number of cards to deal {n_cards} is '
                  f'greater than the available cards {self.undealt_cards()}')
            return cards_to_deal

        # Draw a card from the top of the deck for each requested card
        for i in range(0, n_cards):
            cards_to_deal.append(self.draw())
        return cards_to_deal


class Player(object):
    def __init__(self, name, cards=[]):
        """
        Player constructor
        :param name: String value for the play namer
        :param cards: List of Card objects the play is holding
        """
        self.name = name
        self.cards = cards
        self.pairs = []

    def cards_held(self):
        """
        Cards held
        :return: Integer of the number of cards currently held
        """
        return len(self.cards)

    def pick_card(self, deck):
        """
        Pick a card from the top of the Deck object and add to held cards
        :param deck: Deck object
        :return: Card object picked
        """
        card = deck.draw()
        self.cards.append(card)
        return card

    def add_card(self, card):
        """
        Add card
        :param card: Card object to be add
        :return: N/A
        """
        self.cards.append(card)

    def remove_card(self, card):
        """
        Remove card
        :param card: Card object to remove
        :return: N/A
        """
        self.cards.remove(card)

    def get_random_held_card(self):
        """
        Get random held card (Not removed from the player cards)
        :return: Card object
        """
        card = random.choice(self.cards)
        return card

    def find_pairs(self):
        """
        Find pairs of cards by value
        :return: Boolean value indicating if a pair was found or not
        """
        # Double iteration to find pairs, removing them from cards if found and appending the pair Tuple to pairs
        for card_a in self.cards:
            for card_b in self.cards:
                if card_a != card_b and card_a.value == card_b.value:
                    self.cards.remove(card_a)
                    self.cards.remove(card_b)
                    self.pairs.append((card_a, card_b))
                    return True
        return False

    def get_card_by_value(self, value):
        """
        Get card by value
        :param value: Value to lookup
        :return: Card object if found, otherwise None
        """
        for card in self.cards:
            if card.value == value:
                self.remove_card(card=card)
                return card
        return None

    def get_held_values(self):
        """
        Get held values for all unique values in held cards
        :return: List of strings
        """
        held_values = []
        # Generate a list of all string values for all cards held
        for card in self.cards:
            held_values.append(card.value)
        # Compute the unique set of values
        return list(set(held_values))

In [4]:
print('-' * 20)
print('Welcome to Go Fish!')
print("""
|\   \\\\__     o
| \_/    o \    o 
> _   (( <_  oo  
| / \__+___/      
|/     |/
""")
print('-' * 20)

# Create the Helper object
hlp = Helper()
user_name = hlp.validated_input(input_prompt='Enter your user name', target_type=str)

# Create a deck and shuffle it
deck = Deck()
deck.shuffle()

# Create a user and computer player, starting out with a deal of 7 cards each
player = Player(name=user_name, cards=deck.deal(n_cards=7))
computer = Player(name='computer', cards=deck.deal(n_cards=7))

# Check both for starting pairs
player.find_pairs()
computer.find_pairs()

players_turn = True
turn_count = 0
max_turn_count = 20
while True:
    print('X' * 40)
    print(f'{deck.undealt_cards()} cards left undealt...')
    print(f'PLAYER: \n Cards: {player.cards}\n Held Values: {player.get_held_values()}\n Pairs: {player.pairs}\n\n')
    print(f'COMPUTER: \n Cards: {computer.cards}\n Held Values: {computer.get_held_values()}\n Pairs: {computer.pairs}')
    print('X' * 40)
    print('-' * 80 + '\n\n')
    # Check who has the turn
    if players_turn:
        # Get the player's currently held unique values and prompt what value they want to fish for
        held_values = player.get_held_values()
        player_pick = hlp.validated_input(input_prompt=f'What value do you want to fish for? {held_values}', 
                                          target_type=str, valid_options=held_values)

        print(f'Computer do you have a {player_pick}?')
        # Check if the computer has a card of matching value 
        computer_card = computer.get_card_by_value(value=player_pick)
        if computer_card:
            print(f'Computer - Why yes I do have the {computer_card}\n')
            # If the computer had the card, then add to the cards, check for pairs and restart player turn
            player.add_card(computer_card)
            player.find_pairs()
            players_turn = True
        else:
            print('Go Fish!')
            # The computer did not have the card so pick from the deck and check for pairs
            picked_card = player.pick_card(deck=deck)
            print(f'-- Player picked {picked_card}')
            player.find_pairs()
            
            # If the card picked matches the original value fished then continue the turn
            if picked_card.value == player_pick:
                print('-- Lucky Player picked right!')
                players_turn = True
            else:
                players_turn = False
    else:
        # The computer picks a held card's value to fish for at random
        computer_pick = computer.get_random_held_card()
        choices = [computer_pick.value, 'Go Fish!']
        valid_options = list(set(player.get_held_values()).intersection(set([computer_pick.value]))) + ['Go Fish!']
        
        # Ask the player if they have a card of matching in value
        # (Only allow them to respond with their matching held cards or "Go Fish!")
        player_response = hlp.validated_input(
            input_prompt=f'Player do you have a {computer_pick.value}? {choices}', 
            target_type=str, valid_options=valid_options)
        
        if player_response == 'Go Fish!':
            # The player did not have the card so pick from the deck and check for pairs
            picked_card = computer.pick_card(deck=deck)
            print(f'-- Computer picked {picked_card}')
            computer.find_pairs()

            # If the card picked matches the original value fished then continue the turn
            if picked_card.value == computer_pick:
                print('-- Lucky Computer picked right!')
                players_turn = False
            else:
                players_turn = True
        else:
            # Get the player card of matching value 
            player_card = player.get_card_by_value(value=player_response)
            print(f'Player - Why yes I do have the {player_card}\n')
            # The player had the card so add to the cards, check for pairs and restart computer turn
            computer.add_card(player_card)
            computer.find_pairs()
            players_turn = False
    
    # Break the loop when either the player or computer have run out of cards
    if player.cards_held() == 0 or computer.cards_held() == 0:
        break

    # Break the loop early when max turn count has been reached and declare an early outcome
    if turn_count == 20:
        print('-'*40)
        print(f'Turn count {turn_count} reached!  Finishing early.')
        print('-'*40)
        break
        
    turn_count += 1
        
print('*** GAME OVER ***')
print(f'Final Computer pairs: \n{computer.pairs}\n')
print(f'Final Player pairs: \n{player.pairs}\n')
# Get the final pair counts and determine win, loose or draw outcome
final_computer_pairs = len(computer.pairs)
final_player_pairs = len(player.pairs)
print('RESULT:')
if final_player_pairs > final_computer_pairs:
    print(f'Congratulations, {user_name} you win!')
elif final_player_pairs > final_computer_pairs:
    print(f'Sorry {user_name}, you Lost!')
elif final_player_pairs == final_computer_pairs:
    print('Tie game!')

--------------------
Welcome to Go Fish!

|\   \\__     o
| \_/    o \    o 
> _   (( <_  oo  
| / \__+___/      
|/     |/

--------------------
Enter your user namejking
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
38 cards left undealt...
PLAYER: 
 Cards: [6 of Hearts, 8 of Diamonds, Jack of Clubs, 3 of Hearts, King of Hearts]
 Held Values: ['King', 'Jack', '3', '6', '8']
 Pairs: [(10 of Diamonds, 10 of Clubs)]


COMPUTER: 
 Cards: [Ace of Hearts, 2 of Hearts, 7 of Diamonds, 6 of Diamonds, Queen of Diamonds]
 Held Values: ['Queen', 'Ace', '2', '6', '7']
 Pairs: [(9 of Hearts, 9 of Clubs)]
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
--------------------------------------------------------------------------------


What value do you want to fish for? ['King', 'Jack', '3', '6', '8']6
Computer do you have a 6?
Computer - Why yes I do have the 6 of Diamonds

XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
38 cards left undealt...
PLAYER: 
 Cards: [8 of Diamonds, Jack of Clubs, 3 of Hearts, King of Hea

Player do you have a Ace? ['Ace', 'Go Fish!']Go Fish!
-- Computer picked 10 of Spades
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
30 cards left undealt...
PLAYER: 
 Cards: [8 of Diamonds, Jack of Clubs, 3 of Hearts, King of Diamonds, 4 of Clubs]
 Held Values: ['King', 'Jack', '3', '4', '8']
 Pairs: [(10 of Diamonds, 10 of Clubs), (6 of Hearts, 6 of Diamonds), (King of Hearts, King of Spades), (2 of Clubs, 2 of Hearts)]


COMPUTER: 
 Cards: [Ace of Hearts, Queen of Diamonds, 10 of Spades]
 Held Values: ['Queen', '10', 'Ace']
 Pairs: [(9 of Hearts, 9 of Clubs), (7 of Diamonds, 7 of Spades), (5 of Hearts, 5 of Spades)]
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
--------------------------------------------------------------------------------


What value do you want to fish for? ['King', 'Jack', '3', '4', '8']8
Computer do you have a 8?
Go Fish!
-- Player picked 6 of Clubs
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
29 cards left undealt...
PLAYER: 
 Cards: [8 of Diamonds, Jack of Clubs, 3 of Hear